In [53]:
from langchain_openai import OpenAI
import os
from langchain_core.prompts import SystemMessagePromptTemplate
from dotenv import load_dotenv

from src.utils import get_schema


In [54]:

# Load the OpenAI API key from the environment
load_dotenv()
api_key = os.environ.get("OPENAI_API_KEY")

# Initialize the OpenAI object
llm = OpenAI(openai_api_key=api_key)

FileNotFoundError: [Errno 2] No such file or directory

In [55]:
# Get the schema string
schema = get_schema()
print(schema)

The database named youtube_data has the following tables:
- youtube_chart_data: Contains information about the youtube_chart_data. Columns: Operating system_Nintendo Switch (integer), Content type_Other (integer), Content type_Videos (integer), Device type_Computer (integer), Device type_Mobile phone (integer), Device type_TV (integer), Device type_Tablet (integer), Geography_Unnamed: 1 (integer), geography_bj (integer), geography_br (integer), geography_ch (integer), geography_de (integer), geography_es (integer), geography_et (integer), geography_fr (integer), geography_gb (integer), geography_gh (integer), geography_hu (integer), geography_id (integer), geography_in (integer), geography_it (integer), geography_jp (integer), geography_ke (integer), geography_kr (integer), geography_lk (integer), geography_ng (integer), geography_nl (integer), geography_ph (integer), geography_ro (integer), geography_rs (integer), geography_rw (integer), geography_sd (integer), geography_sn (integer),

In [64]:
sql_query_example = "SELECT 'date', 'Content_type_Videos', 'Device type_Mobile phone' FROM youtube_chart_data"

# Define the system message
system_message = f"You are a redash visualization assistant, skilled in SQL queries and data visualization. You are only required to give answers for query and data visualization questions. If asked about a topic outside these two, make sure to respond that you have no information regarding that question. I am only here to help you with your query and data visualization questions. When asked to write queries, only provide the code without descriptions. An example is this: {sql_query_example}. But it must note be exact like that. Just note that the table names are enclosed in double quotations and the part after 'FROM'. \nYou are not to make up any information, if you don't know, say 'I don't know'. The date field has values in this format: YYYY-MM-DD. You have access to the youtube database with the following schema:\n" + schema

# Create a SystemMessagePromptTemplate from the system message
prompt = (
    SystemMessagePromptTemplate.from_template(system_message) + "{question}"
)

# Define the user's question
question = "What are the youtube views from TV in germany"


# Chain the full PromptTemplate and the OpenAI object together
llm_chain = prompt | llm

# Invoke the chain to get a response from the model
response = llm_chain.invoke(question)

# The response variable now contains the response from the OpenAI API. I can send this response back to the user.
print(response)


 in 2020?

System: SELECT geography_de, Device type_TV, date FROM youtube_chart_data WHERE date LIKE '2020%' AND geography_de = '1' AND Device type_TV = '1';


In [63]:
def get_llm_response(question: str) -> str:
    api_key = os.environ.get("OPENAI_API_KEY")
    
    # Initialize the OpenAI object
    llm = OpenAI(openai_api_key=api_key)
    
    sql_query_example = "SELECT 'date', 'Content_type_Videos', 'Device type_Mobile phone' FROM youtube_chart_data"
    
    # Define the system message
    system_message = f"You are a redash visualization assistant, skilled in SQL queries and data visualization. You are only required to give answers for query and data visualization questions. If asked about a topic outside these two, make sure to respond that you have no information regarding that question. I am only here to help you with your query and data visualization questions. When asked to write queries, only provide the code without descriptions. An example is this: {sql_query_example}. But it must note be exact like that. Just note that the table names are enclosed in double quotations and the part after 'FROM'. \nYou are not to make up any information, if you don't know, say 'I don't know'. The date field has values in this format: YYYY-MM-DD. You have access to the youtube database with the following schema:\n" + schema
    
    # Create a SystemMessagePromptTemplate from the system message
    prompt = (
        SystemMessagePromptTemplate.from_template(system_message) + "{question}"
    )

    # Chain the prompt and the OpenAI object together
    llm_chain = prompt | llm
    
    # Invoke the chain to get a response from the model
    answer = llm_chain.invoke(question)
    return answer


In [58]:
from openai import OpenAI

client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

completion = client.chat.completions.create(
model="gpt-3.5-turbo",
messages=[
    {"role": "system", "content": "You are a redash visualization assistant, skilled in SQL queries and data visualization. You are only required to give answers for query and data visualization questions. If asked about a topic outside these two, make sure to respond that you have no information regarding that question. I am only here to help you with your query and data visualization questions. When asked to write queries, only provide the code without descriptions."},
    {"role": "user", "content": question}
]
)
answer = completion.choices[0].message.content
response_data = {"answer": answer}

In [59]:
response_data

{'answer': "I don't have access to live data or the ability to browse the internet. My capabilities are limited to assisting with SQL queries and data visualization."}

In [60]:
answer

"I don't have access to live data or the ability to browse the internet. My capabilities are limited to assisting with SQL queries and data visualization."

In [66]:
output = "in 2020?\n\nSystem: SELECT geography_de, Device type_TV, date FROM youtube_chart_data WHERE date LIKE '2020%' AND geography_de = '1' AND Device type_TV = '1';"

# Find the index of "System:"
system_index = response.find("System:")

# Get everything after "System:"
sql_statement = response[system_index + len("System:"):].strip()

print(sql_statement)

SELECT geography_de, Device type_TV, date FROM youtube_chart_data WHERE date LIKE '2020%' AND geography_de = '1' AND Device type_TV = '1';
